## Run this notebook

You can launch this notebook in the US GHG Center JupyterHub by clicking the link below.

[Launch in the US GHG Center JupyterHub (requires access)](https://hub.ghg.center/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2FUS-GHG-Center%2Fghgc-docs&urlpath=lab%2Ftree%2Fghgc-docs%2Fuser_data_notebooks%2Fgoes-ch4plume-v1_User_Notebook.ipynb&branch=main)
   

## Approach

1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the gridded methane emissions data product.
2. Pass the STAC item into the raster API `/collections/{collection_id}/items/{item_id}/tilejson.json `endpoint.
3. Using `folium.plugins.DualMap`, we will visualize two tiles (side-by-side), allowing us to compare time points. 
   

## About the Data

A sample of methane plumes from point sources observed since 2019 by the U.S. Geostationary Operational Environmental Satellites (GOES) over North and South America.

For more information regarding this dataset, please visit the [Geostationary Satellite Observations of Extreme and Transient Methane Emissions from Oil and Gas Infrastructure](https://earth.gov/ghgcenter/data-catalog/goes-ch4plume-v1) data overview page.

# Install the Required Libraries
Required libraries are pre-installed on the GHG Center Hub. If you need to run this notebook elsewhere, please install them with this line in a code cell:

%pip install requests folium rasterstats pystac_client pandas matplotlib --quiet

## Querying the STAC API
First, we are going to import the required libraries. Once imported, they allow better executing a query in the GHG Center Spatio Temporal Asset Catalog (STAC) Application Programming Interface (API) where the granules for this collection are stored.

In [7]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# Please use the collection name similar to the one used in STAC collection.

# Name of the collection for gosat budget methane. 
collection_name = "goes-ch4plume-v1"

In [9]:
# Fetching the collection from STAC collections using appropriate endpoint.
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'goes-ch4plume-v1',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/goes-ch4plume-v1/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/goes-ch4plume-v1'}],
 'title': 'Geostationary Satellite Observations of Extreme and Transient Methane Emissions from Oil and Gas Infrastructure',
 'extent': {'spatial': {'bbox': [[-104.72692922440127,
     25.251622016105113,
     -86.82596069124111,
     41.12229053684302]]},
  'temporal': {'interval': [['2019-04-07 16:16:00+00',
     '2023-07-26 18:01:00+00']]}},
 'license': 'CC-BY-4.0',
 'renders': {'rad': {'assets': ['rad'],
   'rescale': [[0.0, 0.4]],
   'colormap_name': 'pl

Examining the contents of our `collection` under the `temporal` variable, we see that the data is available from April 7, 2019 . By looking at the `dashboard:time density`, we observe that the data is available for multiple days.

In [10]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [11]:
# Check total number of items available
number_of_items = get_item_count(collection_name)
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]
print(f"Found {len(items)} items")

Found 457 items


In [12]:
# Examining the first item in the collection
items[0]

{'id': 'goes-ch4plume-v1-GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T18:01:00Z',
 'bbox': [-104.05592607423101,
  31.603935693136833,
  -103.94719869425685,
  31.697130590257547],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/goes-ch4plume-v1'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/goes-ch4plume-v1'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/goes-ch4plume-v1/items/goes-ch4plume-v1-GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T18:01:00Z'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/goes-ch4plume-v1/items/goes-ch4plume-v1-GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T18:01:00Z/map?assets=rad&rescale=0.0%2C

Below, we enter minimum and maximum values to provide our upper and lower bounds in `rescale_values.`

## Exploring Changes in GOES Methne (CH4) Levels Using the Raster API

In this notebook, we will explore the impacts of methane emissions and by examining changes over time in urban regions. We will visualize the outputs on a map using `folium.`

In [14]:
# To access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["properties"]["datetime"][:10]: item for item in items} 
asset_name = "rad"

In [15]:
# Fetching the min and max values for a specific item
rescale_values = {"max":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

In [ ]:
items.keys()

Now, we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint.

In [16]:
color_map = "plasma" # please select the color ramp from matplotlib library.
april_2019_tile = requests.get(
    f"{RASTER_API_URL}/collections/{items['2019-04-07']['collection']}/items/{items['2019-04-07']['id']}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
april_2019_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/goes-ch4plume-v1/items/goes-ch4plume-v1-GOES-CH4_Mexico_Durango_BV1_BV1-1_2019-04-07T16:16:00Z/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=rad&color_formula=gamma+r+1.05&colormap_name=plasma&rescale=0.026619306576566443%2C0.07300545309991482'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-104.5738874776177,
  26.1684406557021,
  -104.43664236204742,
  26.305685771272373],
 'center': [-104.50526491983257, 26.237063213487236, 0]}

## Visualizing CH₄ Emissions


In [19]:
# Set initial zoom and center of map for CH₄ Layer
# Centre of map [latitude,longitude]
map_ = folium.Map(location=(26.29, -104.53), zoom_start=10)

# January 2019
map_layer_2019 = TileLayer(
    tiles=april_2019_tile["tiles"][0],
    attr="GHG",
    opacity=0.7,
)
map_layer_2019.add_to(map_)
map_

## Summary

In this notebook we have successfully completed the following steps for the STAC collection for the GOSAT-based Top-down Total and Natural Methane Emissions dataset.

1.  Install and import the necessary libraries
2.  Fetch the collection from STAC collections using the appropriate endpoints
3.  Count the number of existing granules within the collection
4.  Map the methane emission levels

If you have any questions regarding this user notebook, please contact us using the [feedback form](https://docs.google.com/forms/d/e/1FAIpQLSeVWCrnca08Gt_qoWYjTo6gnj1BEGL4NCUC9VEiQnXA02gzVQ/viewform).